#### This notebook demonstrates the use of the fairness adjuster, using the same structure as the AIF360 adversarial debiasing example

The source notebook can be found here:
https://github.com/Trusted-AI/AIF360/blob/main/examples/demo_adversarial_debiasing.ipynb

In [1]:
%matplotlib inline
# Load all necessary packages
import sys

sys.path.append("../")
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.fairness_adjuster import FairnessAdjuster
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import (
    load_preproc_data_adult,
    load_preproc_data_compas,
    load_preproc_data_german,
)
from aif360.datasets import (
    AdultDataset,
    BinaryLabelDataset,
    CompasDataset,
    GermanDataset,
)
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from IPython.display import Markdown, display
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

tf.disable_eager_execution()

2024-12-05 01:37:33.208856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733362653.220348  134095 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733362653.223844  134095 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 01:37:33.235898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/aif360/lib/python3.11/site-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated

#### Load dataset and set options

In [2]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_german(["age", "sex"])

# privileged_groups = [{'sex': 1}]
# unprivileged_groups = [{'sex': 0}]
privileged_groups = [{"age": 1}]
unprivileged_groups = [{"age": 0}]


dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

/home/ubuntu/AIF360_dev/aif360/algorithms/preprocessing/optim_preproc_helpers/data_preproc_functions.py:261: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['personal_status'].replace(status_map)


In [3]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(
    dataset_orig_train.privileged_protected_attributes,
    dataset_orig_train.unprivileged_protected_attributes,
)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(700, 11)


#### Favorable and unfavorable labels

1.0 2.0


#### Protected attribute names

['age', 'sex']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['age', 'sex', 'credit_history=Delay', 'credit_history=None/Paid', 'credit_history=Other', 'savings=500+', 'savings=<500', 'savings=Unknown/None', 'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed']


#### Metric for original training data

In [4]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(
    dataset_orig_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
display(Markdown("#### Original training dataset"))
print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_orig_train.mean_difference()
)
metric_orig_test = BinaryLabelDatasetMetric(
    dataset_orig_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_orig_test.mean_difference()
)

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.156031
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.134953


In [5]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(
    dataset_orig_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
display(
    Markdown(
        "#### Scaled dataset - Verify that the scaling does not affect the group label statistics"
    )
)
print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_scaled_train.mean_difference()
)
metric_scaled_test = BinaryLabelDatasetMetric(
    dataset_orig_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_scaled_test.mean_difference()
)

#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.156031
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.134953


### Learn plan classifier without debiasing

In [6]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = FairnessAdjuster(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    scope_name="plain_classifier",
    debias=False,
    sess=sess,
)

2024-12-05 01:40:11.702797: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
plain_model.fit(dataset_orig_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch 0; iter: 0; batch classifier loss: 0.691506
epoch 1; iter: 0; batch classifier loss: 0.652496
epoch 2; iter: 0; batch classifier loss: 0.599050
epoch 3; iter: 0; batch classifier loss: 0.645443


I0000 00:00:1733362819.317209  134095 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


epoch 4; iter: 0; batch classifier loss: 0.561317
epoch 5; iter: 0; batch classifier loss: 0.669750
epoch 6; iter: 0; batch classifier loss: 0.556796
epoch 7; iter: 0; batch classifier loss: 0.580640
epoch 8; iter: 0; batch classifier loss: 0.571397
epoch 9; iter: 0; batch classifier loss: 0.574895
epoch 10; iter: 0; batch classifier loss: 0.601938
epoch 11; iter: 0; batch classifier loss: 0.532943
epoch 12; iter: 0; batch classifier loss: 0.546194
epoch 13; iter: 0; batch classifier loss: 0.505287
epoch 14; iter: 0; batch classifier loss: 0.571458
epoch 15; iter: 0; batch classifier loss: 0.539189
epoch 16; iter: 0; batch classifier loss: 0.589610
epoch 17; iter: 0; batch classifier loss: 0.563824
epoch 18; iter: 0; batch classifier loss: 0.550951
epoch 19; iter: 0; batch classifier loss: 0.541019
epoch 20; iter: 0; batch classifier loss: 0.497706
epoch 21; iter: 0; batch classifier loss: 0.486871
epoch 22; iter: 0; batch classifier loss: 0.526006
epoch 23; iter: 0; batch classifier l

In [8]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

2024-12-05 01:40:37.823144: W tensorflow/c/c_api.cc:305] Operation '{name:'plain_classifier/plain_classifier/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node plain_classifier/plain_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](plain_classifier/plain_classifier/classifier_model/b2/Adam_1, plain_classifier/plain_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [9]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(
    dataset_nodebiasing_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_train.mean_difference()
)

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(
    dataset_nodebiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_test.mean_difference()
)

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(
    dataset_orig_test,
    dataset_nodebiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Classification accuracy = %f"
    % classified_metric_nodebiasing_test.accuracy()
)
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5 * (TPR + TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print(
    "Test set: Disparate impact = %f"
    % classified_metric_nodebiasing_test.disparate_impact()
)
print(
    "Test set: Equal opportunity difference = %f"
    % classified_metric_nodebiasing_test.equal_opportunity_difference()
)
print(
    "Test set: Average odds difference = %f"
    % classified_metric_nodebiasing_test.average_odds_difference()
)
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.286140
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.366803


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.666667
Test set: Balanced classification accuracy = 0.531768
Test set: Disparate impact = 0.630165
Test set: Equal opportunity difference = -0.316628
Test set: Average odds difference = -0.371735
Test set: Theil_index = 0.095291


### Apply in-processing algorithm based on adversarial learning

In [10]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [11]:
# Learn parameters with debias set to True
debiased_model = FairnessAdjuster(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    scope_name="debiased_classifier",
    adversary_loss_weight=0.01,
    debias=True,
    sess=sess,
)

In [12]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.712397
epoch 1; iter: 0; batch classifier loss: 0.641707
epoch 2; iter: 0; batch classifier loss: 0.609574
epoch 3; iter: 0; batch classifier loss: 0.601586
epoch 4; iter: 0; batch classifier loss: 0.594361
epoch 5; iter: 0; batch classifier loss: 0.597414
epoch 6; iter: 0; batch classifier loss: 0.623095
epoch 7; iter: 0; batch classifier loss: 0.579439
epoch 8; iter: 0; batch classifier loss: 0.544041
epoch 9; iter: 0; batch classifier loss: 0.500621
epoch 10; iter: 0; batch classifier loss: 0.575007
epoch 11; iter: 0; batch classifier loss: 0.536670
epoch 12; iter: 0; batch classifier loss: 0.536997
epoch 13; iter: 0; batch classifier loss: 0.579212
epoch 14; iter: 0; batch classifier loss: 0.488289
epoch 15; iter: 0; batch classifier loss: 0.532040
epoch 16; iter: 0; batch classifier loss: 0.507235
epoch 17; iter: 0; batch classifier loss: 0.559672
epoch 18; iter: 0; batch classifier loss: 0.541051
epoch 19; iter: 0; batch classifier loss:

In [13]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

2024-12-05 01:41:06.481032: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1, debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [14]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_train.mean_difference()
)
print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_test.mean_difference()
)

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(
    dataset_debiasing_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_debiasing_train.mean_difference()
)

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(
    dataset_debiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_debiasing_test.mean_difference()
)


display(Markdown("#### Plain model - without debiasing - classification metrics"))
print(
    "Test set: Classification accuracy = %f"
    % classified_metric_nodebiasing_test.accuracy()
)
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5 * (TPR + TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print(
    "Test set: Disparate impact = %f"
    % classified_metric_nodebiasing_test.disparate_impact()
)
print(
    "Test set: Equal opportunity difference = %f"
    % classified_metric_nodebiasing_test.equal_opportunity_difference()
)
print(
    "Test set: Average odds difference = %f"
    % classified_metric_nodebiasing_test.average_odds_difference()
)
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())


display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(
    dataset_orig_test,
    dataset_debiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Classification accuracy = %f"
    % classified_metric_debiasing_test.accuracy()
)
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5 * (TPR + TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print(
    "Test set: Disparate impact = %f"
    % classified_metric_debiasing_test.disparate_impact()
)
print(
    "Test set: Equal opportunity difference = %f"
    % classified_metric_debiasing_test.equal_opportunity_difference()
)
print(
    "Test set: Average odds difference = %f"
    % classified_metric_debiasing_test.average_odds_difference()
)
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.286140
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.366803


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.263752
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.223946


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.666667
Test set: Balanced classification accuracy = 0.531768
Test set: Disparate impact = 0.630165
Test set: Equal opportunity difference = -0.316628
Test set: Average odds difference = -0.371735
Test set: Theil_index = 0.095291


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.686667
Test set: Balanced classification accuracy = 0.544405
Test set: Disparate impact = 0.774203
Test set: Equal opportunity difference = -0.090822
Test set: Average odds difference = -0.238832
Test set: Theil_index = 0.071174



    References:
    [1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning," 
    AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.